In [1]:
import pandas as pd
import numpy as np
from glob import glob

df = pd.read_table('/home/groups/dpwall/briannac/y_chromosome_mismappings/data/coverages/chrom_lengths.tsv', header=None)

In [3]:
print('fuck you')

fuck you


In [4]:
df['chrom'] = [i.replace('SN:', '') for i in df[1]]
df['length'] = [int(i.replace('LN:', '')) for i in df[2]]
df.drop([0,1,2], inplace=True, axis=1)

In [5]:
df['start'] = [0] + list(np.cumsum(df['length'])[:-1].values)
df['finish'] = np.cumsum(df['length'])-1
df.index = df.chrom
df = df[['start', 'finish', 'length']]

In [6]:
df.index = ['chrDecoy' if i==25 else idx for i,idx in enumerate(df.index)]
df = df.iloc[:26]
df.head()

,start,finish,length
chr1,0,248956421,248956422
chr2,248956422,491149950,242193529
chr3,491149951,689445509,198295559
chr4,689445510,879660064,190214555
chr5,879660065,1061198323,181538259


In [7]:
df.to_csv('/home/groups/dpwall/briannac/y_chromosome_mismappings/data/coverages/chrom_start_stops.tsv',
         sep='\t')

# Chromosome + file idx mappings

In [8]:
mylist = []
for row in df.iterrows():
    for i in range(3217):
         # If current idx is in middle of chromosome"
        if i==np.floor(row[1].start/1000000): 
            mylist = mylist + [(row[0], '%04d' % i, row[1].start-i*1000000+1, 1000000)]
            #print(row[1].start, i, 'start, so skiprows (1, ', row[1].start-i*1000000+1, ')')
        if i==np.floor(row[1].finish/1000000): 
            mylist = mylist + [(row[0], '%04d' % i, 1, row[1].finish-i*1000000+1)]
            #print(row[1].finish, i, 'finish so only go to row: ', row[1].finish-i*1000000+1)
            break
        elif (i<=np.ceil(row[1].finish/1000000)) & (i>=np.floor(row[1].start/1000000)): 
            #print(i, row[0], 'middle')
            mylist = mylist + [(row[0], '%04d' % i, 1, 1000000)]


In [10]:
new_df = pd.DataFrame(mylist)
new_df.columns = ['chrom', 'file_idx', 'skiprows', 'nrows']
new_df.to_csv('/home/groups/dpwall/briannac/y_chromosome_mismappings/intermediate_files/test.tsv', sep='\t')

In [11]:
new_df

,chrom,file_idx,skiprows,nrows
0,chr1,0000,1,1000000
1,chr1,0000,1,1000000
2,chr1,0001,1,1000000
3,chr1,0002,1,1000000
4,chr1,0003,1,1000000
...,...,...,...,...
3135,chrY,3088,1,269832
3136,chrM,3088,269833,1000000
3137,chrM,3088,1,286401
3138,chrDecoy,3088,286402,1000000


In [12]:
# Unfinsiehd?

In [26]:
import pandas as pd
from glob import glob
files = glob('/home/groups/dpwall/briannac/y_chromosome_mismappings/intermediate_files/covWAS/*.done')
files = [f.split('/')[-1].replace('.done', '') for f in files]

starts_all = pd.read_table('/home/groups/dpwall/briannac/y_chromosome_mismappings/intermediate_files/coverages/chrom_start_stops_intervals.tsv', index_col=0)
starts_all.loc[[i for i in starts_all.index if i not in files]].to_csv(
    '/home/groups/dpwall/briannac/y_chromosome_mismappings/intermediate_files/coverages/chrom_start_stops_intervals_unfinished.tsv',
    sep='\t')